In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
data = pd.read_csv('continuous dataset.csv')

# Convert timestamp to datetime format
data['datetime'] = data['datetime'][:45261].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

# Set the datetime as index
data.set_index('datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['nat_demand'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['nat_demand'].rolling(window=24).mean()
data['rolling_std'] = data['nat_demand'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]/1000
test_data = data[train_size:]/1000

In [5]:
#CNN-LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from keras.losses import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data for CNN-LSTM
X_train = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(64, return_sequences=True))
model.add(Flatten())
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred=model.predict(X_test)

Epoch 1/50
566/566 [==============================] - 10s 15ms/step - loss: 0.0207 - val_loss: 0.0024
Epoch 2/50
566/566 [==============================] - 9s 15ms/step - loss: 0.0022 - val_loss: 0.0020
Epoch 3/50
566/566 [==============================] - 9s 15ms/step - loss: 0.0018 - val_loss: 0.0020
Epoch 4/50
566/566 [==============================] - 8s 15ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 5/50
566/566 [==============================] - 8s 15ms/step - loss: 0.0016 - val_loss: 0.0022
Epoch 6/50
566/566 [==============================] - 8s 15ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 7/50
566/566 [==============================] - 8s 15ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 8/50
566/566 [==============================] - 8s 15ms/step - loss: 0.0014 - val_loss: 0.0013
Epoch 9/50
566/566 [==============================] - 9s 15ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 10/50
566/566 [==============================] - 9s 15ms/step - loss: 0.0013 - val_l

In [6]:
#For MAE
mae = np.mean(np.abs(ypred.ravel() - y_test.ravel()))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
mape = np.mean(np.abs((y_test.ravel() - ypred.ravel()) / y_test.ravel())) * 100
print('MAPE = {:.2f} (%)'.format(mape))


# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

MAE = 0.02 (%)
MAPE = 1.33 (%)
RSME = 0.02 (%)
